In [13]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [2]:
from sentence_transformers import SentenceTransformer

In [7]:
model_name = 'multi-qa-distilbert-cos-v1'
model = SentenceTransformer(model_name)

In [9]:
v = model.encode('I just discovered the course. Can I still join?')

In [11]:
v[0]

0.089264646

In [5]:
len(v)

384

In [29]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [30]:
from tqdm.auto import tqdm

In [31]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

  0%|          | 0/948 [00:00<?, ?it/s]

In [28]:
len(documents[0]['question_text_vector'])

768

In [32]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [33]:
query = 'I just discovered the course. Can I still join it?'

In [34]:
v_q = model.encode(query)

In [35]:
v_q

array([ 7.82226548e-02, -4.01311405e-02,  3.86135913e-02, -1.78966438e-04,
        8.92347097e-02, -5.04591092e-02, -1.05026569e-02,  3.71055678e-02,
       -4.18713912e-02,  3.48084792e-02, -1.20701883e-02, -2.36942340e-02,
        3.87900174e-02,  1.60988607e-02,  3.50747295e-02,  3.04746162e-03,
        5.79672381e-02, -4.10627462e-02, -3.41552682e-02, -2.56396383e-02,
       -3.55263911e-02,  1.42908087e-02, -1.62799917e-02,  3.21446545e-02,
       -4.66897376e-02,  7.89186060e-02,  4.90160920e-02,  1.56761166e-02,
       -1.69110075e-02,  2.26482227e-02,  5.60206100e-02, -3.98361087e-02,
        6.77409917e-02, -1.20209912e-02,  1.12621894e-03, -1.94394365e-02,
       -2.65951678e-02,  1.06177367e-02,  1.69687122e-02,  1.13487840e-02,
       -2.97063086e-02,  5.25258258e-02, -1.41453547e-02,  4.61699851e-02,
        1.17066065e-02, -2.38053519e-02, -6.32558241e-02, -1.92042235e-02,
       -7.10592186e-03,  3.24167833e-02,  2.49618199e-02, -5.27503015e-03,
        2.01149024e-02, -

In [36]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [37]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

In [38]:
import pandas as pd

In [40]:
#df_ground_truth = pd.read_csv('ground-truth-data.csv')

In [41]:
#ground_truth = df_ground_truth.to_dict(orient='records')

In [42]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [43]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [44]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [45]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [57]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

In [46]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [47]:
evaluate(ground_truth, question_vector_knn)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.8076502732240437, 'mrr': 0.6985519125683067}

ES text only: 0.7395720769397017, 0.6032418413658963

In [48]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('text_vector', v_q, course)

In [49]:
evaluate(ground_truth, text_vector_knn)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.8803278688524591, 'mrr': 0.7658925318761388}

In [50]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

evaluate(ground_truth, question_text_vector_knn)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}

In [51]:
def elastic_search_knn_combined(vector, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "course": course
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'text_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_text_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [52]:
def vector_combined_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, course)

evaluate(ground_truth, vector_combined_knn)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9218579234972678, 'mrr': 0.8335974499089249}

In [55]:
q = {}
q['question'] = query
q['course'] = 'machine-learning-zoomcamp'

q

{'question': 'I just discovered the course. Can I still join it?',
 'course': 'machine-learning-zoomcamp'}

#Q5: Indexing with Elasticsearch.

In [56]:
vector_combined_knn(q)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

#Q4 Hit-rate for our search engine

In [58]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

NameError: name 'X' is not defined